In [ ]:
import math
import torch

device = torch.device("cpu")

embed_dim = 256
n_head = 8


In [ ]:
class Embedding(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(Embedding, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
    def forward(self, x):
        out = self.embed(x)
        return out

In [ ]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, max_seq_len, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.embed_dim = embed_dim
        pe = torch.zeros(max_seq_len, self.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, self.embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (i / self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** (((i + 1)) / self.embed_dim)))
        #pe = pe.unsqueeze(0)
        pe = pe.T
        self.register_buffer('pe', pe)       # pre-calculate and freeze

    def forward(self, x):
        seq_len = x.size(1)
        return x * math.sqrt(self.embed_dim) + torch.autograd.Variable(self.pe[:seq_len], requires_grad=False)
        # return torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.single_head_dim = self.embed_dim // self.num_heads   # 512 / 8 = 64, every head takes its part of the input
        self.query_matrix = torch.nn.Linear(self.embed_dim, self.embed_dim, bias=False)  # a big matrix for the whole input
        self.key_matrix = torch.nn.Linear(self.embed_dim, self.embed_dim, bias=False)    # we will split the heads later on
        self.value_matrix = torch.nn.Linear(self.embed_dim, self.embed_dim, bias=False)
        self.mixer = torch.nn.Linear(self.embed_dim, self.embed_dim)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V shapes are (batch x length x single_head_dim) == (32 x 10 x 64)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.single_head_dim)
        # scores shape is (batch x length x length) == (32 x 10 x 10)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)        # put large negative numbers to positions where mask == 0
        probs = torch.softmax(scores, dim=-1)                   # transform scores to probabilities (B x L x L)
                                                                # for each sample in the batch tell how ith token is important to jth token
        return torch.matmul(probs, V)                           # re-weigh values according to the probabilities

    def split_heads(self, x):
        batch_size, seq_length, embed_dim = x.size()            # B x L x D
        assert self.embed_dim == embed_dim
        return x.view(                                          # B x L x H x d
            batch_size,
            seq_length,
            self.num_heads,
            self.single_head_dim,
        ).transpose(1, 2)                                       # B x H x L x d
        
    def combine_heads(self, x):
        batch_size, num_heads, seq_length, single_head_dim = x.size()
        assert self.num_heads == num_heads
        assert self.single_head_dim == single_head_dim
        # B x H x L x d   => 
        # B x L x H x d   =>
        # B x L x D
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.embed_dim)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.query_matrix(Q))
        K = self.split_heads(self.key_matrix(K))
        V = self.split_heads(self.value_matrix(V))        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        return self.mixer(self.combine_heads(attn_output))

In [ ]:
class FeedForward(torch.nn.Module):
    def __init__(self, embed_dim, expansion):
        super(FeedForward, self).__init__()
        self.fc1 = torch.nn.Linear(embed_dim, embed_dim * expansion)
        self.fc2 = torch.nn.Linear(embed_dim * expansion, embed_dim)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        # expand => non-linearity => shrink
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, embed_dim, num_heads, expansion, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads)
        self.feed_forward = FeedForward(embed_dim, expansion)
        self.norm1 = torch.nn.LayerNorm(embed_dim)
        self.norm2 = torch.nn.LayerNorm(embed_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, mask):
        # (attention => droupout) + residual => norm
        x = self.norm1(x + self.dropout(self.self_attn(x, x, x, mask)))
        # (MLP => dropout) + residual => norm
        return self.norm2(x + self.dropout(self.feed_forward(x)))

In [ ]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, embed_dim, num_heads, expansion, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(embed_dim, num_heads)
        self.cross_attn = MultiHeadAttention(embed_dim, num_heads)
        self.feed_forward = FeedForward(embed_dim, expansion)
        self.norm1 = torch.nn.LayerNorm(embed_dim)
        self.norm2 = torch.nn.LayerNorm(embed_dim)
        self.norm3 = torch.nn.LayerNorm(embed_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # self attention (decoder x decoder)
        x = self.norm1(x + self.dropout(self.self_attn(x, x, x, tgt_mask)))
        # cross attention (decoder x encoder), note that queries are from decoder, keys and values - from encoder
        x = self.norm2(x + self.dropout(self.cross_attn(x, encoder_output, encoder_output, src_mask)))
        # MLP + dropout + residual + norm
        return self.norm3(x + self.dropout(self.feed_forward(x)))

In [ ]:
class EncoderDecoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, expansion, max_seq_length, dropout):
        super(EncoderDecoder, self).__init__()
        # we could split encoder and decoder embeddings
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = PositionalEncoding(embed_dim, max_seq_length)
        self.encoder_layers = torch.nn.ModuleList([EncoderLayer(embed_dim, num_heads, expansion, dropout) for _ in range(num_layers)])
        self.decoder_layers = torch.nn.ModuleList([DecoderLayer(embed_dim, num_heads, expansion, dropout) for _ in range(num_layers)])
        self.fc = torch.nn.Linear(embed_dim, vocab_size)
        self.dropout = torch.nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.embedding(tgt)))
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        output = self.fc(dec_output)
        return output

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, expansion, max_seq_length, dropout):
        super(Encoder, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.positional_encoding = PositionalEncoding(embed_dim, max_seq_length)
        self.layers = torch.nn.ModuleList([EncoderLayer(embed_dim, num_heads, expansion, dropout) for _ in range(num_layers)])
        self.fc = torch.nn.Linear(embed_dim, vocab_size)
        self.dropout = torch.nn.Dropout(dropout)

    def generate_mask(self, src):
        mask = (src != 0).unsqueeze(1).unsqueeze(3)
        seq_length = src.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        mask = mask & nopeak_mask
        return mask

    def forward(self, src):
        mask = self.generate_mask(src)
        embedded = self.dropout(self.positional_encoding(self.embedding(src)))
        for layer in self.layers:
            embedded = layer(embedded, mask)
        return self.fc(embedded)

In [ ]:
import requests

all_shakespeare = requests.get("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt").content.decode()
print(len(all_shakespeare))

In [ ]:
dictionary = ["<empty>", "<start>", "<end>"] + list(set(all_shakespeare))
print(dictionary)
print(len(dictionary))

sym2idx = {s: i for i, s in enumerate(dictionary)}
print(sym2idx)

In [ ]:
import random

random.seed(42)

data = all_shakespeare.split("\n\n")
data = list(filter(lambda x: x and len(x) < 1000, data))
random.shuffle(data)

print(len(data))
print(data[77])

In [ ]:
train = [data[i] for i in range(len(data)) if i % 10 != 0]
test = [data[i] for i in range(len(data)) if i % 10 == 0]

print(train[-5])
print("")
print(test[-5])

In [ ]:
src = torch.tensor([[1, 2, 3, 0]])
model = Encoder(len(dictionary), 128, 4, 4, 4, 1000, 0.1)
mask = model.generate_mask(src)
print(mask)
print(model(src))

In [ ]:
import numpy as np

def generate(model, len_limit):
    model.eval()
    with torch.no_grad():
        result = ["<start>"]
        while len(result) < len_limit:
            x = torch.tensor([[sym2idx[x] for x in result]]).to(device)
            y = model(x)
            y = y[0][0].cpu().numpy()
            y = np.exp(y)
            y /= np.sum(y)
            x = dictionary[np.random.choice(y.shape[0], p = y)]
            if x in ["<start>", "<end>", "<empty>"]:
                break
            result.append(x)
        return result

print(generate(model, 1000))

In [ ]:
import tqdm

def iterate_batches(data, batch_size, device):
    x, y, max_len = [], [], 0
    for k in tqdm.tqdm(range(len(data))):
        item = data[k]
        x.append([sym2idx[sym] for sym in ["<start>"] + list(item)])
        y.append([sym2idx[sym] for sym in list(item) + ["<end>"]])
        max_len = max(max_len, len(x[-1]))
        if len(x) == batch_size or k + 1 == len(data):
            for i in range(len(x)):
                x[i] = x[i] + [sym2idx["<empty>"] for _ in range(max_len - len(x[i]))]
                y[i] = y[i] + [sym2idx["<empty>"] for _ in range(max_len - len(y[i]))]
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)
            yield x, y
            x, y, max_len = [], [], 0
        

def train_epoch(data, model):
    model.train()
    loss_function = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    optimizer = torch.optim.AdamW(model.parameters())
    total_loss, total_count = 0.0, 1e-38
    random.shuffle(data)
    for inputs, answers in iterate_batches(data, 16, device):
        optimizer.zero_grad()
        #print(inputs.shape)
        #print(answers.shape)
        outputs = model(inputs)
        outputs = outputs.transpose(1, 2)
        #print(outputs.shape)
        loss = loss_function(outputs, answers)
        total_loss += (loss.item() * inputs.shape[0])
        total_count += inputs.shape[0]
        loss.backward()
        optimizer.step()
    return total_loss / total_count

def test_epoch(data, model):
    with torch.no_grad():
        model.eval()
        loss_function = torch.nn.CrossEntropyLoss()
        total_loss, total_count = 0.0, 1e-38
        for inputs, answers in iterate_batches(data, 16, device):
            outputs = model(inputs)
            outputs = outputs.transpose(1, 2)
            loss = loss_function(outputs, answers)
            total_loss += (loss.item() * inputs.shape[0])
            total_count += inputs.shape[0]
        return total_loss / total_count

for i in range(100):
    train_loss = train_epoch(train, model)
    test_loss = test_epoch(test, model)
    print("Epoch {} loss: {:.5f} {:.5f}".format(i, train_loss, test_loss))
    print(generate(model, 1000))
    print("")